In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By


In [2]:
import pandas as pd
import time

In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
df = pd.read_excel('D:/ap_cust/SPTDATA.xlsx',header=None)
for column in df.columns:
    li = df[column].tolist()

In [ ]:
len(li)

In [6]:
ap_cust1 = []

In [73]:

for entry in range(8000,len(li)):
    cid = li[entry]
    driver = webdriver.Chrome()
    driver.get('https://www.apeasternpower.com/viewBillDetailsMain')
    id = driver.find_element(By.ID, 'ltscno')
    id.send_keys(cid)
    sum = driver.find_element(By.ID, 'Billquestion').text.split()
    a = sum[0]
    b = sum[2].split('=')[0]
    sum = int(a) + int(b)
    ans = driver.find_element(By.ID, 'Billans')
    ans.send_keys(sum)
    WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "Billsignin"))
    ).click()

    time.sleep(5)
    soup = BeautifulSoup(driver.page_source,'lxml')
    name = soup.find_all('td')[6].text.strip()
    # print(name)
    
    # time.sleep(5)
    driver.execute_script('window.scrollBy(0, 280)')
    # time.sleep(5)
    WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "historyDivbtn"))
        ).click()

    time.sleep(2)
    soup = BeautifulSoup(driver.page_source,'lxml')
    table = soup.find('table',{'id':'consumptionData'}).text.strip().split('\n')

    headers = table[:4]
    # time.sleep(5)
    table1 = soup.find('tbody')
    
    table1 = table1.text.strip().split('\n')
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source,'lxml')
    tables = soup.find('table',{'id':'consumptionData'})

    amt = tables.find_all('input', {'class': 'btn btn-link'})
    amount = []
    for i in range(len(amt)):
        amount.append(amt[i]['value'])

    headers = table[:4]
    list = []
    i = 0
    j = 0
    while i < len(table1):
        data = {}
        data[headers[0]]= table1[i]
        data[headers[1]] = table1[i+1]
        data[headers[2]] = table1[i+2]
        data['amount'] = amount[j]
        j += 1
        i+=14
        list.append(data)    

    customer_list = {}
    
    customer_list['Name'] = name
    customer_list['Scon'] = cid
    customer_list['Transactions'] = list
    ap_cust1.append(customer_list)
    driver.quit()    




In [74]:
len(ap_cust1)

999

In [75]:
import json

with open(f'ap_cust1.json', 'w') as f:
    json.dump(ap_cust1, f, indent=4)